-----

# **Multidataframs**

----

- In langchain, agents can access many dataframes at once.

## **Install Required Packages**

In [ ]:
!pip install langchain langchain-experimental langchain-community watermark openai langchain-openai

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.6 MB

## **Import Required Libraries**

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import OpenAI

## **Set Up OpenAI API Key**

In [ ]:
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

## **Read Data**

In [ ]:
url = "https://raw.githubusercontent.com/adamerose/datasets/master/titanic.csv"
df=pd.read_csv(url)
print(df.shape)
df.head()

(891, 15)


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### **Agent to Interact With Single Dataframe**

In [ ]:
llm=OpenAI()

In [ ]:
# Creating an agent that can interact with a Pandas DataFrame using a language model (llm)
# The agent is initialized with the DataFrame (df) and configured with the following options:
# - verbose: If set to True, the agent will provide detailed output during its operation
# - allow_dangerous_code: If set to True, the agent is permitted to execute code that could be potentially unsafe

agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code=True)

In [ ]:
agent.invoke("How many rows are there")



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe
Action: python_repl_ast
Action Input: len(df)891891 is the number of rows in the dataframe
Final Answer: There are 891 rows in the dataframe.

> Finished chain.


{'input': 'How many rows are there',
 'output': 'There are 891 rows in the dataframe.'}

In [ ]:
agent.invoke("How many people have more than 23 age")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe for rows where age is greater than 23 and then count the number of rows
Action: python_repl_ast
Action Input: len(df[df['age'] > 23])468468 people have more than 23 age
Final Answer: 468

> Finished chain.


{'input': 'How many people have more than 23 age', 'output': '468'}

## **Multi DataFrame Example**

- Agent can also interact with Multiple DataFrames passed in a list

In [ ]:
df1=df.copy() # making copy of orignal nata

In [ ]:
df1["age"] = df1["age"].fillna(df1["age"].mean()) # impute missing values in df1 in age column

In [ ]:
# Creating an agent that can interact with multiple Pandas DataFrames using a language model (llm)
# The agent is initialized with a list of DataFrames ([df, df1]) and configured with the following options:
# - verbose: If set to True, the agent will provide detailed output during its operation
# - allow_dangerous_code: If set to True, the agent is permitted to execute code that could be potentially unsafe

agent = create_pandas_dataframe_agent(llm, [df, df1], verbose=True, allow_dangerous_code=True)

In [ ]:
agent.invoke("How many rows in the age column are different")



> Entering new AgentExecutor chain...
Thought: I should count the number of unique values in the age column
Action: python_repl_ast
Action Input: df1['age'].nunique()8888 unique values in the age column
Action: python_repl_ast
Action Input: df2['age'].nunique()8989 unique values in the age column
Action: python_repl_ast
Action Input: df1['age'].nunique() - df2['age'].nunique()-188 - 89 = -1, so there is one more unique value in df2 compared to df1
Final Answer: There is 1 more unique value in the age column of df2 compared to df1.

> Finished chain.


{'input': 'How many rows in the age column are different',
 'output': 'There is 1 more unique value in the age column of df2 compared to df1.'}